In [1]:
import os

In [2]:
%pwd

'h:\\Projects\\mlops\\first\\pizza-steak-end-to-end\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'h:\\Projects\\mlops\\first\\pizza-steak-end-to-end'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareCallbackConfig:
    root_dir: Path
    tensorboard_log_dir: Path
    checkpoint_dir: Path



In [6]:
from PizzaSteakClassifier.constants import *
from PizzaSteakClassifier.utils.common import read_yaml_file, create_directories

In [7]:
class ConfiguationManager:
    def __init__(self,
                 config_path: Path = CONFIG_FILE_PATH):
        
        self.config = read_yaml_file(config_path)
        create_directories([self.config.artifacts_root])
    
    def get_prepare_callback_config(self) -> PrepareCallbackConfig:
        config = self.config

        create_directories([
            Path(config.prepare_callbacks.tensorboard_log_dir),
            Path(config.prepare_callbacks.checkpoint_dir)
        ])

        prepare_callback_config = PrepareCallbackConfig(
            root_dir = Path(config.artifacts_root),
            tensorboard_log_dir = Path(config.prepare_callbacks.tensorboard_log_dir),
            checkpoint_dir = Path(config.prepare_callbacks.checkpoint_dir)
        )

        return prepare_callback_config
    


In [8]:
import tensorflow as tf
import time

[2024-01-04 20:29:35,629 - WARNING - module_wrapper - From h:\Projects\mlops\first\pizza-steak-end-to-end\.venv\Lib\site-packages\keras\src\losses.py:2976: The name tf.losses.sparse_softmax_cross_entropy is deprecated. Please use tf.compat.v1.losses.sparse_softmax_cross_entropy instead.
]


In [9]:
type(CONFIG_FILE_PATH)

pathlib.WindowsPath

In [10]:
type(os.path.join(CONFIG_FILE_PATH, "hello"))

str

In [13]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbackConfig):
        self.config = config
    
    @property
    def create_tb_callback(self):
        timestamp = time.strftime("%Y%m%d-%H%M%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_log_dir,
            f"tb_logs_{timestamp}"
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    
    @property
    def create_ckpt_callback(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=str(self.config.checkpoint_dir), #str is required for ModelCheckpoint
            save_best_only=True
        )
    
    def get_tb_ckpt_callbacks(self):
        return [
                self.create_tb_callback,
                self.create_ckpt_callback
        ]

In [14]:
try:
    config=ConfiguationManager()
    prepare_callback_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callback_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()
    
except Exception as e:
    raise e


[2024-01-04 20:50:17,527 - INFO - common - YAML file loaded successfully: config\config.yaml]
[2024-01-04 20:50:17,529 - INFO - common - Directory created: artifacts]
[2024-01-04 20:50:17,531 - INFO - common - Directory created: artifacts\prepare_callbacks\tensorboard_logs]
[2024-01-04 20:50:17,533 - INFO - common - Directory created: artifacts\prepare_callbacks\checkpoint_dir\model.h5]
